# Possible New Geometry - Waffle Chart

It could be an alternative to the pie charts. It is only proof of concept, it is not in the library now.

Data for the notebook originally comes from [here](https://data.world/datanerd/cat-vs-dog-popularity-in-u-s).

In [1]:
from io import BytesIO
import requests

import numpy as np
import pandas as pd
from PIL import Image

from lets_plot import *
LetsPlot.setup_html()

from geoms.waffle import *

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/cats_vs_dogs.csv')

In [3]:
cat_cols = ['Location']
num_cols = ['Dog Owning Households (1000s)', 'Cat Owning Households']
df = df[cat_cols + num_cols]
for column in num_cols:
    df[column] = df[column].str.replace(' ', '').astype(int)

In [4]:
def get_classifier(*, alpha=1.0/3.0):
    if alpha < 0 or alpha >= 1:
        raise Exception('Bad alpha: %f' % alpha)
    def classifier(data):
        ratio = data['Dog Owning Households (1000s)'] / data['Cat Owning Households']
        data['More Popular Pet Type'] = 'Cat' if ratio <= (1 - alpha) / (1 + alpha) \
                                              else ('Dog' if ratio >= (1 + alpha) / (1 - alpha) else 'Both')
        return data
    return classifier

df['More Popular Pet Type'] = None
df = df.apply(get_classifier(alpha=.05), axis=1)
df.head()

,Location,Dog Owning Households (1000s),Cat Owning Households,More Popular Pet Type
0,Alabama,807,501,Dog
1,Arizona,1008,743,Dog
2,Arkansas,550,351,Dog
3,California,4260,3687,Dog
4,Colorado,845,642,Dog


In [5]:
combined_df = df.groupby('More Popular Pet Type')['Location'].count().reset_index()
combined_df.columns = ['More Popular Pet Type', 'Count']
combined_df

,More Popular Pet Type,Count
0,Both,13
1,Cat,5
2,Dog,31


## Chart by Default

In [6]:
ggplot() + \
    geom_waffle(aes(value='Count', group='More Popular Pet Type', fill='More Popular Pet Type'), combined_df) + \
    ggtitle('Dog and Cat Popularity in US') + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

## Chart with Changed Sizes

In [7]:
ggplot() + \
    geom_waffle(aes(value='Count', group='More Popular Pet Type', fill='More Popular Pet Type'), \
                combined_df, chart_shape=(16, 12), cell_shape=(.8, .6), color='black', size=1) + \
    ggtitle('Dog and Cat Popularity in US') + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

## Customize Shape and Coloring

In [8]:
grid = np.matrix([
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
])

ggplot() + \
    geom_waffle(aes(value='Count', group='More Popular Pet Type', fill='More Popular Pet Type'), \
                combined_df, grid=grid) + \
    scale_fill_manual(values=['#e41a1c', '#377eb8', '#4daf4a']) + \
    ggtitle('Dog and Cat Popularity in US') + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

## Example of Using JPEG Mask for More Complicated Chart Shape

In [9]:
MASK_WIDTH, MASK_HEIGHT = 40, 40
cat_mask_data = requests.get('https://raw.githubusercontent.com'
                             '/HIL-HK/lets-plot-examples/master/data/images/cat_mask.jpeg')
cat_mask = np.array(Image.open(BytesIO(cat_mask_data.content)).resize((MASK_WIDTH, MASK_HEIGHT), Image.BILINEAR))
grid = [[(0 if color.mean() > 255 / 2 else 1) for color in row] for row in cat_mask]

ggplot() + \
    geom_waffle(aes(value='Count', group='More Popular Pet Type', fill='More Popular Pet Type'), \
                combined_df, grid=grid) + \
    scale_fill_manual(values=['#800026', '#fc4e2a', '#fed976']) + \
    ggtitle('Dog and Cat Popularity in US') + \
    ggsize(600, 450) + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')